In [ ]:
!pip install -qU langchain langchain-community langchain-text-splitters
!pip install -qU langchain-openai langchain-chroma pypdf langchain_experimental
!pip install -qU PyPDF2 rank_bm25
!pip install -qU FlagEmbedding transformers crewai
!pip install -qU pyngrok
!pip install -qU flask_cors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.4/644.4 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/9

# **Set up LLM API key**

In [ ]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# **1. Data preprocessing**

## **1.1. Extract PDF**

In [ ]:
pdf_path = "data.pdf"

In [ ]:
from PyPDF2 import PdfReader

def extract_text_with_pypdf2(pdf_path):
    """Extract text from a PDF file using PyPDF2."""
    text = ""
    reader = PdfReader(pdf_path)
    for page in reader.pages:
        text += page.extract_text()
    return text

In [ ]:
text = extract_text_with_pypdf2(pdf_path)
text

' \n58 CÔNG BÁO/S ố 1215 + 1216/Ngày 27-11-2023  \n  \nCỘNG HÒA XÃ H ỘI CHỦ NGHĨA VIỆT NAM \nĐộc lập - Tự do - Hạnh phúc \n \nLUẬT \nGIAO THÔNG ĐƯỜ NG BỘ \n \nLuật Giao thông đường bộ số 23/2008/QH12 ngày 13 tháng 11 n ăm 2008 c ủa \nQuốc hội, có hiệu lực kể từ ngày 01 tháng 7 n ăm 2009, được sửa đổi, bổ sung bởi: \n1. Luật số 35/2018/QH14 ngày 20 tháng 11 n ăm 2018 c ủa Quốc hội sửa đổi, \nbổ sung một số điều của 37 luật có liên quan đến quy hoạch, có hiệu lực kể từ ngày 01 \ntháng 01 n ăm 2019; \n2. Luật Phòng, ch ống tác hại của rượu, bia số 44/2019/QH14 ngày 14 tháng 6 \nnăm 2019 của Quốc hội, có hiệu lực kể từ ngày 01 tháng 01 n ăm 2020. \n3. Luật Giá số 16/2023/QH15 ngày 19 tháng 6 n ăm 2023 c ủa Quốc hội, có \nhiệu lực kể từ ngày 01 tháng 7 n ăm 2024. \nCăn cứ Hiến pháp nước Cộng hòa xã h ội chủ nghĩa Việt Nam năm 1992 đã \nđược sửa đổi, bổ sung một số điều theo Ngh ị quyết số 51/2001/QH10; \nQuốc hội ban hành Lu ật Giao thông đường bộ1. \n                                       

## **1.2. Create Semantic Chunk**

In [ ]:
import json
import re
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def preprocess_and_chunk_text(text):
    """
    Preprocess the text and split it into chunks with titles and contexts.
    - Titles include both the current chapter and article.
    - Contexts contain the text under each article.
    """
    # Define regex patterns for identifying chapters and articles
    chapter_pattern = r"(Chương\s+[IVXLCDM]+\s*[\n\r]*[A-ZÀÁẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬĐÈÉẺẼẸÊẾỀỂỄỆÌÍỈĨỊÒÓỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÙÚỦŨỤƯỨỪỬỮỰ\s]+?)(?=\s*Điều|$)"
    article_pattern = r"(Điều\s+\d+[a-z]?\.\s*[A-ZÀÁẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬĐÈÉẺẼẸÊẾỀỂỄỆÌÍỈĨỊÒÓỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÙÚỦŨỤƯỨỪỬỮỰ][^\n]+)"

    # Split the text into sections based on chapters and articles
    sections = re.split(f"({chapter_pattern}|{article_pattern})", text, flags=re.DOTALL)

    # Initialize variables for processing
    current_chapter = None
    current_article = None
    chunks = []
    buffer = ""

    for section in sections:
        # Skip None or empty sections
        if section is None or not section.strip():
            continue

        # Check if the section is a chapter title
        chapter_match = re.match(chapter_pattern, section)
        if chapter_match:
            # If there's a previous article, save its content as a chunk
            if current_article and buffer.strip():
                chunk = {
                    "title": f"{current_article} {current_chapter}",
                    "context": buffer.strip()
                }
                chunks.append(chunk)

            # Update the current chapter
            current_chapter = section.strip()
            current_article = None  # Reset article when a new chapter starts
            buffer = ""  # Reset buffer for new chapter
            continue

        # Check if the section is an article title
        article_match = re.match(article_pattern, section)
        if article_match:
            # If there's a previous article, save its content as a chunk
            if current_article and buffer.strip():
                chunk = {
                    "title": f"{current_article} {current_chapter}",
                    "context": buffer.strip()
                }
                chunks.append(chunk)

            # Update the current article
            current_article = section.strip()
            buffer = ""  # Reset buffer for new article
            continue

        # If it's neither a chapter nor an article, it's part of the current article's content
        if current_article:
            buffer += " " + section.strip()

    # Add the last chunk if there's any remaining content
    if current_article and buffer.strip():
        chunk = {
            "title": f"{current_article} {current_chapter}",
            "context": buffer.strip()
        }
        chunks.append(chunk)

    return chunks

def split_long_context(title, context, max_length=800):
    """
    Split long context into smaller chunks using RecursiveCharacterTextSplitter.
    Each smaller chunk retains the same title.
    """
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=max_length,
        chunk_overlap=300,  # Overlap to ensure continuity between chunks
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    sub_chunks = splitter.split_text(context)
    return [{"title": title, "context": sub_chunk.strip()} for sub_chunk in sub_chunks]

def process_pdf(pdf_path, output_json):
    """Process the PDF and save the output as JSON."""
    # Step 1: Extract text from the PDF using the specified method
    raw_text = extract_text_with_pypdf2(pdf_path)

    # Debug: Print first 500 characters of extracted text
    print(f"Extracted text (first 500 chars): {raw_text[:500]}")

    # Step 2: Preprocess and chunk the text
    chunks = preprocess_and_chunk_text(raw_text)

    # Step 3: Split long contexts into smaller chunks
    final_chunks = []
    for chunk in chunks:
        title = chunk["title"]
        context = chunk["context"]
        if len(context) > 800:  # If context is too long, split it
            sub_chunks = split_long_context(title, context)
            final_chunks.extend(sub_chunks)
        else:
            final_chunks.append(chunk)

    # Debug: Print number of chunks created
    print(f"Number of chunks created: {len(final_chunks)}")
    print(final_chunks)

    # Step 4: Save the chunks to a JSON file
    save_to_json(final_chunks, output_json)

def save_to_json(data, output_file):
    """Save the processed data to a JSON file."""
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
process_pdf(pdf_path, "output.json")

Extracted text (first 500 chars):  
58 CÔNG BÁO/S ố 1215 + 1216/Ngày 27-11-2023  
  
CỘNG HÒA XÃ H ỘI CHỦ NGHĨA VIỆT NAM 
Độc lập - Tự do - Hạnh phúc 
 
LUẬT 
GIAO THÔNG ĐƯỜ NG BỘ 
 
Luật Giao thông đường bộ số 23/2008/QH12 ngày 13 tháng 11 n ăm 2008 c ủa 
Quốc hội, có hiệu lực kể từ ngày 01 tháng 7 n ăm 2009, được sửa đổi, bổ sung bởi: 
1. Luật số 35/2018/QH14 ngày 20 tháng 11 n ăm 2018 c ủa Quốc hội sửa đổi, 
bổ sung một số điều của 37 luật có liên quan đến quy hoạch, có hiệu lực kể từ ngày 01 
tháng 01 n ăm 2019; 
2. Luật Phò
Number of chunks created: 156
[{'title': 'Điều 1. Phạm vi điều chỉnh Chương I \nNHỮNG QUY ĐỊNH CHUNG', 'context': 'Luật này quy định về quy tắc giao thông đường bộ; kết cấu hạ tầng giao thông \nđường bộ; phương tiện và người tham gia giao thông đường bộ; vận tải đường bộ \nvà quản lý nhà n ước về giao thông đường bộ.'}, {'title': 'Điều 2. Đối tượng áp dụng Chương I \nNHỮNG QUY ĐỊNH CHUNG', 'context': 'Luật này áp d ụng đối với tổ chức, cá nhân liên quan đến gia

In [ ]:
import json

with open('/kaggle/working/output.json', 'r', encoding='utf-8') as file:
    json_data = json.load(file)

processed_documents = []
for item in json_data:
    combined_text = f"Trích dẫn ở: {item['title']} \n Nội dung như sau: {item['context']}"
    processed_documents.append(combined_text)


In [ ]:
processed_documents

['Trích dẫn ở: Điều 1. Phạm vi điều chỉnh Chương I \nNHỮNG QUY ĐỊNH CHUNG \n Nội dung như sau: Luật này quy định về quy tắc giao thông đường bộ; kết cấu hạ tầng giao thông \nđường bộ; phương tiện và người tham gia giao thông đường bộ; vận tải đường bộ \nvà quản lý nhà n ước về giao thông đường bộ.',
 'Trích dẫn ở: Điều 2. Đối tượng áp dụng Chương I \nNHỮNG QUY ĐỊNH CHUNG \n Nội dung như sau: Luật này áp d ụng đối với tổ chức, cá nhân liên quan đến giao thông đường bộ \ntrên lãnh th ổ nước Cộng hòa xã h ội chủ nghĩa Việt Nam.',
 'Trích dẫn ở: Điều 3. Giải thích từ ngữ Chương I \nNHỮNG QUY ĐỊNH CHUNG \n Nội dung như sau: Trong Lu ật này, các t ừ ngữ dưới đây được hiểu như sau: \n1. Đường bộ gồm đường, cầu đường bộ, hầm đường bộ, bến phà đường bộ. \n2. Công trình đường bộ gồm đường bộ, nơi dừng xe, đỗ xe trên đường bộ, đèn \ntín hiệu, biển báo hiệu, vạch kẻ đường, cọc tiêu, rào ch ắn, đảo giao thông, d ải phân \ncách, cột cây số, tường, kè, hệ thống thoát n ước, trạm kiểm tra tải trọng xe

In [ ]:
from langchain_core.documents import Document

documents = [Document(page_content=text) for text in processed_documents]

# **2. Retriever Module**

## **2.1. Create Vector Database**

In [ ]:
current_dir = "/kaggle/working"
persistent_directory = os.path.join(current_dir, "db", "chroma_db_pdf")

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

def create_vector_store(documents, db_path):
    """
    Creates a Chroma vector store from chunked documents.

    Parameters:
    chunks (List[Document]): A list of chunked Document objects.
    db_path (str): The directory path to persist the vector store.

    Returns:
    Chroma: A Chroma vector store containing the embedded documents.
    """

    print("Chroma vector store is created...\n")
    embedding_model = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1024)
    db = Chroma.from_documents(documents=documents, embedding=embedding_model, persist_directory=db_path)

    return db

In [ ]:
db = create_vector_store(documents, persistent_directory)

Chroma vector store is created...



/tmp/ipykernel_31/378124050.py:17: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1024)
/usr/local/lib/python3.11/dist-packages/langchain_community/embeddings/openai.py:271: UserWarning: WARNING! dimensions is not default parameter.
                    dimensions was transferred to model_kwargs.
                    Please confirm that dimensions is what you intended.
  warnings.warn(


## **2.2. Semantic Retriever**

In [ ]:
semantic_retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 5})
relevant_documents = semantic_retriever.invoke("Nếu tôi vượt đèn đỏ thì tôi sẽ bị phạt gì?")

In [ ]:
relevant_documents

[Document(metadata={}, page_content='Trích dẫn ở: Điều 8. Các hành vi b ị nghiêm c ấm Chương I \nNHỮNG QUY ĐỊNH CHUNG \n Nội dung như sau: điều khiển xe tham gia giao thông đường bộ. \n11. Điều khiển xe cơ giới chạy quá tốc độ quy định, giành đường, vượt ẩu. \n12. Bấm còi, rú ga liên t ục; bấm còi trong th ời gian từ 22 giờ đến 5 giờ, bấm \ncòi hơi, sử dụng đèn chiếu xa trong đô thị và khu đông dân c ư, trừ các xe được \nquyền ưu tiên đang đi làm nhiệm vụ theo quy định của Luật này. \n13. Lắp đặt, sử dụng còi, đèn không đúng thiết kế của nhà sản xuất đối với \ntừng loại xe cơ giới; sử dụng thiết bị âm thanh gây m ất trật tự an toàn giao thông, \ntrật tự công cộng. \n                                           \n4 Khoản này được sửa đổi, bổ sung theo quy định tại khoản 1 Điều 35 của Luật Phòng, \nchống tác hại của rượu, bia số 44/2019/QH14, có hi ệu lực kể từ ngày 01 tháng 01 n ăm 2020.  \n66 CÔNG BÁO/S ố 1215 + 1216/Ngày 27-11-2023'),
 Document(metadata={}, page_content='Trích dẫn ở: Đi

## **2.3. BM25 Retriever**

In [ ]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(documents, k=5)

In [ ]:
bm25_results = bm25_retriever.invoke("Vượt đèn đỏ")
bm25_results

[Document(metadata={}, page_content='Trích dẫn ở: Điều 25.  Đi trên đoạn đường bộ giao nhau cùng m ức với đường sắt, cầu Chương II  \nQUY TẮC GIAO THÔNG ĐƯỜ NG BỘ \n Nội dung như sau: bộ phải dừng lại phía phần đường của mình và cách rào ch ắn một khoảng cách an \ntoàn; khi đèn tín hiệu đã tắt, rào chắn mở hết, tiếng chuông báo hi ệu ngừng mới \nđược đi qua. \n3. Tại nơi đường bộ giao nhau cùng m ức với đường sắt chỉ có đèn tín hiệu \nhoặc chuông báo hi ệu, khi đèn tín hiệu mầu đỏ đã bật sáng ho ặc có tiếng chuông \nbáo hiệu, người tham gia giao thông đường bộ phải dừng ngay l ại và giữ khoảng \ncách tối thiểu 5 mét tính t ừ ray gần nhất; khi đèn tín hiệu đã tắt hoặc tiếng chuông \nbáo hiệu ngừng mới được đi qua. \n4. Tại nơi đường bộ giao nhau cùng m ức với đường sắt không có đèn tín hiệu, \nrào chắn và chuông báo hi ệu, người tham gia giao thông đường bộ phải quan sát c ả \nhai phía, khi th ấy chắc chắn không có ph ương tiện đường sắt đang đi tới mới được'),
 Document(metadata={}, pa

## **2.4. Reranker**

In [ ]:
from FlagEmbedding import FlagReranker

reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True)

2025-04-12 15:35:14.520769: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744472114.933329      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744472115.061308      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

# **3. Retriever**

## **3.1. Retrieval Module using Hybrid Search and Reranking**

In [ ]:
class Retriever:
  def __init__(self, semantic_retriever, bm25_retriever, reranker):
    self.semantic_retriever = semantic_retriever
    self.bm25_retriever = bm25_retriever
    self.reranker = reranker

  def __call__(self, query):
    semantic_results = self.semantic_retriever.invoke(query)
    bm25_results = self.bm25_retriever.invoke(query)

    content = set()
    for doc in semantic_results:
      content.add(doc.page_content)
    for doc in bm25_results:
      content.add(doc.page_content)
    content = list(content)

    # pairs = [[query, c] for c in content]
    # scores = self.reranker.compute_score(pairs, normalize=True)
    # print(scores)

    # relevant_documents = []
    # for i, score in enumerate(scores):
    #   if score > 0.6:
    #     relevant_documents.append(content[i])

    # return relevant_documents
    return content


In [ ]:
retriever = Retriever(semantic_retriever, bm25_retriever, reranker)

In [ ]:
rs = retriever("Tôi 17 tuổi thì có thể sử dụng xe phân khối bao nhiêu?")
rs

['Trích dẫn ở: Điều 43.  Phạm vi đất dành cho đường bộ Chương III  \nKẾT CẤU HẠ TẦNG GIAO THÔNG ĐƯỜ NG BỘ \n Nội dung như sau: lang an toàn đường bộ thì được tiếp tục sử dụng đất theo đúng mục đích đã được xác \nđịnh và không được gây cản trở cho việc bảo vệ an toàn công trình đường bộ.  \nTrường hợp việc sử dụng đất gây ảnh hưởng đến việc bảo vệ a n  t o à n  c ô n g  \ntrình đường bộ thì chủ công trình và ng ười sử dụng đất phải có biện pháp kh ắc \nphục, nếu không kh ắc phục được thì Nhà n ước thu hồi đất và bồi thường theo quy \nđịnh của pháp luật. \n5. Chính ph ủ quy định cụ thể phạm vi đất dành cho đường bộ, việc sử dụng, \nkhai thác đất hành lang an toàn đường bộ và việc xây dựng các công trình thi ết yếu \ntrong phạm vi đất dành cho đường bộ.',
 'Trích dẫn ở: Điều 58.  Điều kiện của người lái xe tham gia giao thông Chương V  \nNGƯỜI ĐIỀU KHIỂN PHƯƠNG TIỆN \nTHAM GIA GIAO THÔNG ĐƯỜ NG BỘ \n Nội dung như sau: 1. Người lái xe tham gia giao thông ph ải đủ độ tuổi, sức khỏe quy định

## **3.2. Retrieval Tool**

In [ ]:
from typing import Type, Any, Dict
from crewai.tools import BaseTool
from pydantic import BaseModel, Field

class RetrieveLawInput(BaseModel):
    """Schema input for law query"""
    query: str = Field(..., description="Câu hỏi pháp luật cần truy vấn")

class RetrieveLawTool(BaseTool):
    name: str = "RetrieveLaw"
    description: str = (
        "Công cụ dùng để truy vấn tài liệu pháp luật Việt Nam bằng kỹ thuật kết hợp "
        "giữa semantic search và BM25. Dành riêng cho các câu hỏi về điều luật, mức phạt, quy định, v.v."
    )
    args_schema: Type[BaseModel] = RetrieveLawInput

    def _run(self, query: str) -> Any:
        results = retriever(query)
        return "\n\n".join(results[:5])

In [ ]:
retriever_tool = RetrieveLawTool()
# retriever_tool.run("Tôi 17 tuổi thì có thể sử dụng xe phân khối bao nhiêu?")

# **4. Agent**

In [ ]:
from crewai import Agent, Task, Crew, LLM

llm = LLM(model="gpt-4o")

## **4.1. Define Agent**

In [ ]:
# Rewrite Agent
rewrite_agent = Agent(
    role="Chuyên gia phân tích ý định người dùng",
    goal="Hiểu rõ mục đích tin nhắn và diễn đạt lại theo cách rõ ràng, tối ưu cho truy vấn tìm kiếm",
    backstory="""
    Bạn là một chuyên gia trong việc phân tích ngữ nghĩa và mục đích thực sự của người dùng trong các hệ thống hỏi đáp, đặc biệt là trong lĩnh vực pháp lý.
    Bạn có khả năng viết lại tin nhắn một cách ngắn gọn, rõ ràng và độc lập, giúp cải thiện đáng kể độ chính xác của hệ thống tìm kiếm thông tin.
    """,
    llm=llm,
    allow_delegation=False,
    verbose=True
)

# Router Agent
router_agent = Agent(
    role="Chuyên gia phân luồng truy vấn",
    goal="Kiểm tra xem câu hỏi có cần truy xuất cơ sở dữ liệu pháp lý hay không",
    backstory="Chuyên gia phân tích câu hỏi và trả lời có nếu cần truy xuất đến các nguồn dữ liệu pháp lý hoặc trả lời không nếu câu hỏi không liên quan đến pháp luật.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

# Retriever Agent
retriever_agent = Agent(
    role="Truy xuất thông tin pháp lý",
    goal="Truy xuất thông tin pháp lý nếu câu hỏi yêu cầu",
    backstory="""Có kinh nghiệm trong việc truy xuất thông tin pháp lý từ nhiều nguồn như cơ sở dữ liệu.""",
    llm=llm,
    tools=[retriever_tool],
    allow_delegation=False,
    verbose=True
)

# Answer Agent
answer_agent = Agent(
    role="Tạo câu trả lời",
    goal="Tạo ra câu trả lời rõ ràng và chính xác",
    backstory="""Có khả năng tổng hợp và trả lời câu hỏi một cách chính xác, ngắn gọn và dễ hiểu, dù câu hỏi liên quan đến pháp luật hay không.""",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

## **4.2. Define Agent Task**

In [ ]:
# Task for Rewriter Agent
rewrite_task = Task(
    description="""
    Dựa trên đoạn hội thoại trước đó giữa người dùng và trợ lý, hãy xác định ý định thực sự mà người dùng muốn truyền đạt trong lượt chat cuối cùng.
    Viết lại nội dung đó thành một câu hỏi hoặc phát ngôn rõ ràng, ngắn gọn và có thể hiểu độc lập (không phụ thuộc vào lịch sử chat).
    Giữ nguyên dạng gốc của câu cuối cùng (nghi vấn, khẳng định hoặc phủ định).
    Đoạn hội thoại đầu vào:
    {chat_history}
    """,
    expected_output="""
    Một nội dung đã được viết lại rõ ràng và dễ truy xuất, phản ánh đúng mục đích giao tiếp của người dùng.
    Kết quả là dạng string.
    """,
    agent=rewrite_agent
)

# Task for Router Agent
router_task = Task(
    description="""
    Phân tích câu hỏi đã được viết lại và xác định xem nó có liên quan đến pháp luật về giao thông hay không.
    Nếu có, trả về 'truy_xuat' để tiếp tục truy xuất cơ sở dữ liệu pháp luật. Nếu không, trả về 'khong_truy_xuat'.
    """,
    expected_output="""
    Một chuỗi ký tự: 'truy_xuat' nếu câu hỏi là về pháp luật, hoặc 'khong_truy_xuat' nếu là trò chuyện thông thường.
    Kết quả phải viết thường và chính xác.
    """,
    agent=router_agent,
    context=[rewrite_task]
)

# Task for Retriever Agent
retriever_task = Task(
    description="""
    Nếu kết quả từ router là 'truy_xuat', hãy thực hiện truy xuất dữ liệu pháp luật liên quan đến câu hỏi đã viết lại với tham số truyền vào là câu hỏi được viết lại từ rewrite_agent.
    Ngược lại, không thực hiện gì và trả về một chuỗi rỗng "".
    """,
    expected_output="Một văn bản pháp luật hoặc chuỗi rỗng nếu không cần truy xuất.",
    agent=retriever_agent,
    context=[rewrite_task, router_task]
)

# Task for Answer Agent
answer_task = Task(
    description="""
    Tạo câu trả lời phù hợp cho người dùng dựa trên câu hỏi đã được viết lại và có giải thích, kết quả phân luồng và (nếu cần) kết quả truy xuất dữ liệu.
    Nếu kết quả từ router là 'truy_xuat', sử dụng dữ liệu từ hệ thống truy xuất pháp luật để trả lời.
    Nếu kết quả là 'khong_truy_xuat', trả lời trực tiếp dựa trên kiến thức của mô hình.
    """,
    expected_output="""
    Một đoạn văn bản trả lời rõ ràng, chính xác, và thân thiện và có giải thích. Câu trả lời phải đúng với nội dung pháp luật nếu có truy xuất,
    hoặc mang tính trò chuyện tự nhiên nếu không liên quan pháp luật.
    """,
    agent=answer_agent,
    context=[rewrite_task, retriever_task, router_task]  # Có thể thêm retriever_task nếu bạn tích hợp lại
)

## **4.3. Define Crew**

In [ ]:
agentic_rag_crew = Crew(
    agents=[rewrite_agent, router_agent, retriever_agent, answer_agent],
    tasks=[rewrite_task, router_task, retriever_task, answer_task],
    verbose=False,
)

In [ ]:
chat_history = [
    {"role": "user", "content": "Chào cậu"},
    {"role": "assistant", "content": "Chào bạn! Tôi có thể giúp gì về luật giao thông ạ?"},
    {"role": "user", "content": "Tôi 17 tuổi thì có thể sử dụng xe phân khối bao nhiêu?"}
]
history_string = "\n".join(
    [f"{message['role']} : {message['content']}" for message in chat_history]
)

In [ ]:
result = agentic_rag_crew.kickoff(inputs={"chat_history":history_string})

15:53:13 - LiteLLM:INFO: utils.py:2896 - 
LiteLLM completion() model= gpt-4o; provider = openai


# Agent: Chuyên gia phân tích ý định người dùng
## Task: 
    Dựa trên đoạn hội thoại trước đó giữa người dùng và trợ lý, hãy xác định ý định thực sự mà người dùng muốn truyền đạt trong lượt chat cuối cùng.
    Viết lại nội dung đó thành một câu hỏi hoặc phát ngôn rõ ràng, ngắn gọn và có thể hiểu độc lập (không phụ thuộc vào lịch sử chat).
    Giữ nguyên dạng gốc của câu cuối cùng (nghi vấn, khẳng định hoặc phủ định).
    Đoạn hội thoại đầu vào:
    user : Chào cậu
assistant : Chào bạn! Tôi có thể giúp gì về luật giao thông ạ?
user : Tôi 17 tuổi thì có thể sử dụng xe phân khối bao nhiêu?
    


15:53:14 - LiteLLM:INFO: utils.py:1084 - Wrapper: Completed Call, calling success_handler
15:53:14 - LiteLLM:INFO: utils.py:2896 - 
LiteLLM completion() model= gpt-4o; provider = openai




# Agent: Chuyên gia phân tích ý định người dùng
## Final Answer: 
Tôi 17 tuổi có thể lái xe phân khối bao nhiêu được phép theo quy định?


# Agent: Chuyên gia phân luồng truy vấn
## Task: 
    Phân tích câu hỏi đã được viết lại và xác định xem nó có liên quan đến pháp luật về giao thông hay không.
    Nếu có, trả về 'truy_xuat' để tiếp tục truy xuất cơ sở dữ liệu pháp luật. Nếu không, trả về 'khong_truy_xuat'.
    


15:53:15 - LiteLLM:INFO: utils.py:1084 - Wrapper: Completed Call, calling success_handler
15:53:15 - LiteLLM:INFO: utils.py:2896 - 
LiteLLM completion() model= gpt-4o; provider = openai


...



# Agent: Truy xuất thông tin pháp lý
## Thought: Thought: Câu hỏi yêu cầu truy xuất thông tin pháp luật về độ tuổi và phân khối xe máy mà người 17 tuổi có thể lái theo quy định.
## Using tool: RetrieveLaw
## Tool Input: 
"{\"query\": \"Ng\\u01b0\\u1eddi 17 tu\\u1ed5i c\\u00f3 th\\u1ec3 l\\u00e1i xe c\\u00f3 dung t\\u00edch xi lanh bao nhi\\u00eau theo quy \\u0111\\u1ecbnh ph\\u00e1p lu\\u1eadt Vi\\u1ec7t Nam?\"}"
## Tool Output: 
Trích dẫn ở: Điều 58.  Điều kiện của người lái xe tham gia giao thông Chương V  
NGƯỜI ĐIỀU KHIỂN PHƯƠNG TIỆN 
THAM GIA GIAO THÔNG ĐƯỜ NG BỘ 
 Nội dung như sau: 1. Người lái xe tham gia giao thông ph ải đủ độ tuổi, sức khỏe quy định tại 
Điều 60 của Luật này và  có giấy phép lái xe phù h ợp với loại xe được phép điều 
khiển do cơ quan nhà n ước có thẩm quyền cấp. 
Người tập lái xe ô tô khi tham gia giao thông ph ải thực hành trên xe t ập lái và 
có giáo viên b ảo trợ tay lái. 
2. Người lái xe khi điều khiển phương tiện phải mang theo các gi ấy tờ sau: 
a) Đă

15:53:19 - LiteLLM:INFO: utils.py:1084 - Wrapper: Completed Call, calling success_handler
15:53:19 - LiteLLM:INFO: utils.py:2896 - 
LiteLLM completion() model= gpt-4o; provider = openai




# Agent: Truy xuất thông tin pháp lý
## Final Answer: 
Theo quy định pháp luật Việt Nam, người đủ 16 tuổi trở lên được phép lái xe gắn máy có dung tích xi-lanh dưới 50 cm³. Vì vậy, người 17 tuổi có thể lái xe gắn máy có dung tích xi-lanh dưới 50 cm³.


# Agent: Tạo câu trả lời
## Task: 
    Tạo câu trả lời phù hợp cho người dùng dựa trên câu hỏi đã được viết lại và có giải thích, kết quả phân luồng và (nếu cần) kết quả truy xuất dữ liệu.
    Nếu kết quả từ router là 'truy_xuat', sử dụng dữ liệu từ hệ thống truy xuất pháp luật để trả lời.
    Nếu kết quả là 'khong_truy_xuat', trả lời trực tiếp dựa trên kiến thức của mô hình.
    


15:53:20 - LiteLLM:INFO: utils.py:1084 - Wrapper: Completed Call, calling success_handler




# Agent: Tạo câu trả lời
## Final Answer: 
Theo quy định pháp luật Việt Nam, người từ 16 tuổi trở lên được phép điều khiển xe gắn máy có dung tích xi-lanh dưới 50 cm³. Như vậy, bạn 17 tuổi có thể lái xe có dung tích xi-lanh dưới 50 cm³ một cách hợp pháp.




In [ ]:
result.raw

'Một người 17 tuổi được phép sử dụng xe gắn máy có dung tích xi-lanh dưới 50 cm³ theo quy định tại Điều 60 của Luật Giao thông đường bộ Việt Nam.'

# **5. API**

In [ ]:
import os
import json
from google.colab import userdata
from pyngrok import ngrok
from flask import Flask, jsonify, request
from flask_cors import CORS

In [ ]:
authtoken = userdata.get("ngrok_key")
ngrok.set_auth_token(authtoken)

In [ ]:
from flask import Flask, request, jsonify

# authtoken = userdata.get("ngrok_key")
# ngrok.set_auth_token(authtoken)

app = Flask(__name__)
CORS(app)

@app.route('/chat', methods=['POST'])
def chat_v2():
    user_message = request.json.get('message', '')
    context = request.json.get('context', [])

    print(f"Message: {user_message}")
    print(f"Context: {context}")

    history_string = "\n".join(
        [f"{message['role']} : {message['content']}" for message in context]
    )

    try:
        response = agentic_rag_crew.kickoff(inputs={"chat_history": history_string + f"\nuser : {user_message}"})
        return jsonify({"response": response.raw})
    except Exception as e:
        print("Lỗi khi xử lý agentChain:", e)
        return jsonify({"response": "Xin lỗi, đã xảy ra lỗi khi xử lý yêu cầu của bạn."}), 500

if __name__ == '__main__':
    # Start ngrok to tunnel the Flask app
    url = ngrok.connect(5000)
    print(f" * ngrok tunnel: {url}")

    # Start Flask app
    app.run(port=5000)

 * ngrok tunnel: NgrokTunnel: "https://18ff-35-232-78-45.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off
